In [11]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import sys

from osgeo import osr, ogr, gdal
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import math
import time
import pandas as pd
from scipy import interpolate
import json
from matplotlib import colors as mcolors
import matplotlib.image as mpimg
import matplotlib
from shapely import geometry
import time 
import pyproj
from shapely.affinity import rotate
from shapely.geometry import Point, LineString, Polygon
plt.ion()
%matplotlib qt
matplotlib.use("Qt5Agg")

sys.path.append('/media/sf_VBox_Shared/PAOLA/PRISMA3/')

from PRISMA_SDS import PRIS_tools_v2, PRIS_img_v2, PRIS_profiles_v2, PRIS_Shoreline_v2

**DEFINE THE FUNCTIONS**

In [13]:
def GridRaster(dir_img, crs):
    
    """
    Stores the image cells into a pandas geodatabase (cells as polygon features)
    """
    
    raster = gdal.Open(dir_img)
    geoT = raster.GetGeoTransform()
    pixWidth = geoT[1]
    n_cols = raster.RasterXSize
    n_rows = raster.RasterYSize
    
    row = []
    col = []
    pol = []
    
    for r in range(n_rows):
        for c in range(n_cols):
            
            Xcoord, Ycoord = PRIS_tools_v2.pixel_to_world(raster, c, r)
            upper_left = [Xcoord, Ycoord]
            bottom_left = [Xcoord, Ycoord - pixWidth]
            upper_right = [Xcoord + pixWidth, Ycoord]
            bottom_right = [Xcoord + pixWidth, Ycoord - pixWidth]
            
            poly = geometry.Polygon([bottom_left, upper_left, upper_right, bottom_right, bottom_left])
            
            pol.append(poly)
            row.append(r)
            col.append(c)
            
    dataframe = pd.DataFrame({'row':row, 'col':col, 'geometry':pol})
    grid = gpd.GeoDataFrame(dataframe, crs = crs)
    
    return grid

**ITERATE THROUGH THE PROFILES SEARCHING THE INTERFACE PIX**

In [14]:
def ProfileBounds(pr):
    
    """
    Returns the profile BB
    """
    
    xmin = pr.bounds['minx'].values[0]
    xmax = pr.bounds['maxx'].values[0]
    ymin = pr.bounds['miny'].values[0]
    ymax = pr.bounds['maxy'].values[0]
    
    return xmin, xmax, ymin, ymax

def ProfileBeginning(pr):
    
    """
    Gives the profile starting coordinate
    """
    
    pr.reset_index(inplace = True)
    coords = [(coords) for coords in list(pr.geometry[0].coords)]
    x1 = coords[0][0]
    y1 = coords[0][1]
    
    return x1, y1

def EuclideanDistance(x1, y1, x2, y2):
    
    """
    Calculates the euclidean distance between 2 points
    
    """
 
    dis_X = (x1 - x2)**2
    dis_Y = (y1 - y2)**2
    dist = np.sqrt(dis_X + dis_Y)

    return dist

def MidPointDistances(overlayed, x_prx, y_prx):
    
    """
    Obtain the midpoint of a segment
    
    """
    
    distances = []

    for point in overlayed['midpoint']:

        mid_point = list(point.coords)
        x2, y2 = mid_point[0][0], mid_point[0][1]

        distances.append(EuclideanDistance(x_prx, y_prx, x2 ,y2))
        
    overlayed['dist_midpoint'] = distances
    overlayed.sort_values(by = 'dist_midpoint', ascending = True, inplace = True)
    
    return overlayed

In [15]:
def MeanReflectances(overlayed, arr):
    
    """
    Add to the overlated geodatabase the mean reflectance value of the particular pixel
    """
    
    mean_reflectance = []

    for r, c in zip(overlayed['row'], overlayed['col']):
        ss = arr[r,c,:]
        mean_reflectance.append(np.mean(ss))

    overlayed['mean_R'] = mean_reflectance
    
    return overlayed

def Interpolate_R_PR(overlayed, interval):
    
    x_old = np.array(overlayed['dist_midpoint'])
    y_old = np.array(overlayed['mean_R'])
    
    x_new = np.arange(x_old[0], x_old[-1], interval)
    f = interpolate.interp1d(x_old, y_old, kind = 'cubic')
    y_new =  f(x_new)
    
    deriv_new = y_new[1:] - y_new[:-1]
    idd_inter_new = np.argmin(deriv_new)
    inter = y_new[idd_inter_new]
    
    return idd_inter_new, inter, x_new, y_new, deriv_new

def SSInterfacePixe(GeoT, arr, lon, lat):

    # Pixel containing the transition coordinate
    
    col, row = PRIS_tools_v2.world_to_pixel(GeoT, lon, lat)
    
    # Get the spectral signature
    
    ss = arr[row, col, :]
    
    return col, row, ss
    
    

def PlotProfileValues(paths, date, idd_inter_new, inter, x_new, y_new, deriv_new, pr):

        
    pathS0 = os.path.join(paths['Interface_pix_plot'], 'MeanSS_pr')
    
    if not os.path.exists(pathS0):
        os.mkdir(pathS0)
    
    
    name = 'MeanSS_'+ pr + '_' + date + '.png'

    pathS = os.path.join(pathS0, date)

    if not os.path.exists(pathS):
        os.mkdir(pathS)


        
    plt.scatter(idd_inter_new, inter, s=150, c='firebrick', alpha = 0.5, label = 'interface')
    plt.plot(y_new, marker = 'o', color='royalblue', label = 'mean reflectance') # royalblue
    plt.plot(deriv_new, marker = 'o', color = 'slategray', label = 'derived') # #  skyblue

    ax.spines['top'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_color('grey')
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('grey')
    ax.grid(color='gainsboro', linestyle = '--', axis = 'y')

    plt.title('Profile '+ pr, fontsize = 15)
    plt.xlabel("d (m)", fontsize = 15)
    plt.ylabel("Mean Reflectance", fontsize = 15)
    plt.legend()
    plt.savefig(os.path.join(pathS, name))
    plt.show()
    plt.close()
    

**FUNCTIONS RELATED TO THE PROFILE DIRECTION**

In [16]:
def AzimuthAngle(coor_pr_i, coor_pr_f):
    angle = (180/np.pi) * math.atan2(coor_pr_f[0] - coor_pr_i[0], coor_pr_f[1] -coor_pr_i[1])
    
    if angle < 0:
        angle = (angle + 360)%360
    return(angle)

In [17]:
def dot(vA, vB):
    return vA[0]*vB[0] + vA[1] * vB[1]

def Cal_ang(lineA, lineB):
    vA = [(lineA[0][0] - lineA[1][0]), (lineA[0][1] - lineA[1][1])]
    vB = [(lineB[0][0] - lineB[1][0]), (lineB[0][1] - lineB[1][1])]

    # Get dot profuct
    dot_pro = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_pro/magA/magB
    
    # Get angle in radians and then convert to degrees
    if dot_pro == 0:

        ang_deg = np.degrees(np.arccos(0)) 
    else:
        angle = math.acos(dot_pro/magB/magA)
        # Doing angle <- angle mod 360
        ang_deg = math.degrees(angle)%360

    if ang_deg - 180 >= 0:
        return 360 - ang_deg
    else:
        return ang_deg

In [18]:
# This function is used only to check if a profiles is fully contained on the image BB

def ImageBB(paths, date):
    
    imgName = [f for f in os.listdir(paths['PanSharp_RGB']) if f.endswith('.tiff') and date in f][0]
    img = gdal.Open(os.path.join(paths['PanSharp_RGB'], imgName))

    nC = img.RasterXSize
    nR = img.RasterYSize

    geoT = img.GetGeoTransform()
    res = geoT[1]
    xmin = min(geoT[0], geoT[0] + nC * geoT[1])
    xmax = max(geoT[0], geoT[0] + nC * geoT[1])
    ymin = min(geoT[3], geoT[3] + nR * geoT[5])
    ymax = max(geoT[3], geoT[3] + nR * geoT[5])
    north = [(xmin, ymin), (xmin, ymax)]
    
    return xmin, xmax, ymin, ymax, north

In [19]:
def Save_plotInfo(paths, date, pr, idd_inter_new, inter, x_new, y_new, deriv_new ):
    
    filename = '{:s}_MeanR_{:d}_{:.20f}.txt'.format(pr,idd_inter_new, inter)
    pathS0 = paths['Interface_pix_txtPlot']
    
    pathS = os.path.join(pathS0, date)

    if not os.path.exists(pathS):
        os.mkdir(pathS)
    
    dir_fileout = os.path.join(pathS, filename)
    
    #fileout = open(dir_fileout, "w")
    
    b = deriv_new[-1]
    
    deriv_new2 = np.append(deriv_new,b)
    
    info = pd.DataFrame({'x':x_new, 'y':y_new, 'deriv':deriv_new2})
    info.to_csv(dir_fileout, index = False)
    
    #for i in range(len(x_new)):

        #fileout.write('{:9.3f} {:9.3f} {:9.3f}\n'.format(x_new[i], y_new[i], deriv_new2[i]))
                         
    #fileout.close()
        

In [20]:

def AnaliziyngProfiles(profiles, date, grid, arr, img_BB, GeoT, plot):
    
    # Generate the DataFrame

    lon = []
    lat = []
    
    cols = []
    rows = []
    SS = []
    PRS = []

    for pr in profiles['PR']:
        

        print("", end=f"\r Analyzing {pr}")
        
        # Search the profile

        prx = profiles[profiles['PR'] == pr]
        prx.reset_index(inplace = True, drop = True)
        
        if not img_BB.contains(prx['geometry'][0]):
            continue

        # Get pixel initial coordinate

        x_prx, y_prx = ProfileBeginning(prx)
        
        x1, y1 = prx.boundary[0][0].xy[0][0], prx.boundary[0][0].xy[1][0]
        x2, y2 = prx.boundary[0][1].xy[0][0], prx.boundary[0][1].xy[1][0]
        col1, row1 = PRIS_tools_v2.world_to_pixel(GeoT, x1, y1)
        col2, row2 = PRIS_tools_v2.world_to_pixel(GeoT, x2, y2)

        rowmin = min(row1, row2)
        rowmax = max(row1, row2)
        colmin = min(col1, col2)
        colmax = max(col1, col2)

        new_grid = grid[(grid['row'] >= rowmin) & (grid['row']<=rowmax) & (grid['col'] <= colmax) & (grid['col'] >= colmin)]
        new_grid.reset_index(inplace = True, drop = True)


        # Overlay the grid and the profiles geometries

        overlayed = gpd.overlay(prx, new_grid, how = 'intersection')
        overlayed.reset_index(inplace = True, drop = True)

        ## Check that the length of the obtained pixels is the same of the entire profile
        

        assert [np.abs(overlayed['geometry'].length.sum() - prx['geometry'].length) < 1.e-6][0][0]

        ## Check that all the distances are smaller than 5*sqrt(2)

        overlayed['length'] =  [line.length for line in overlayed['geometry']]
        overlayed['bool'] = overlayed['length'] < 5 * np.sqrt(2)
        

        assert overlayed['bool'].unique()[0]

        ## Obtain the mid point of the segments

        overlayed['midpoint'] = overlayed.apply(lambda row: row['geometry'].centroid, axis=1) #Find centroid

        ## Calculate the distances of each midpoint to the beginning profile point

        overlayed = MidPointDistances(overlayed, x_prx, y_prx)

        ## Add the mean reflectances
        

        overlayed = MeanReflectances(overlayed, arr)
        overlayed.to_csv(os.path.join(paths['Interface_pix_txtPlot'], pr + '_Overlayed.csv'))

        ## Interpolate distances an reflectances

        idd_inter_new, inter, x_new, y_new, deriv_new = Interpolate_R_PR(overlayed, 0.25)
        
        Save_plotInfo(paths, date, pr, idd_inter_new, inter, x_new, y_new, deriv_new )

        ## Save and generate the plots
        
        if plot:

            PlotProfileValues(paths, date, idd_inter_new, inter, x_new, y_new, deriv_new, pr) 

        # Now we know which is the point we can obtain the shoreline

        new_length = x_new[idd_inter_new]

        # We have to know which is the profile angle to be able to 
        
        xmin_pr, xmax_pr, ymin_pr, ymax_pr = ProfileBounds(prx)
        
        north = [(xmin_pr, ymin_pr), (xmin_pr, ymax_pr)] # nortth direction
        
        coords = [(coords) for coords in list(prx.geometry[0].coords)]
        coor_pr_i, coor_pr_f = [coords[i] for i in (0,-1)]
        slp = (coor_pr_f[1] - coor_pr_i[1]) / (coor_pr_f[0] - coor_pr_i[0]) # slope profile
        pr_line = [(coor_pr_i[0], coor_pr_i[1]), (coor_pr_f[0], coor_pr_f[1])]
        
        angle = AzimuthAngle(coor_pr_i, coor_pr_f)
        Deg_rot = Cal_ang(north, pr_line)
        
        start = Point(coor_pr_i[0], coor_pr_i[1])
        end = Point(start.x  , start.y + new_length)
        Line = LineString([start, end])
        

        if (90 < angle <270) and (slp < 0):

            pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

        if (180 < angle <270) and (slp > 0):

            pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

        if (0 < angle <90) and (slp > 0):

            pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)
            
        if (270< angle < 360) and (slp < 0):
            
            pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)
        
        
        coords_n = [(coords) for coords in list(pr_new.coords)]
        coor_pr_iN, coor_pr_fN = [coords_n[i] for i in (0,-1)]
        
        col, row, ss = SSInterfacePixe(GeoT, arr, coor_pr_fN[0], coor_pr_fN[1])
        
        cols.append(col)
        rows.append(row)
        SS.append(list(ss))
        PRS.append(pr)

        lon.append(coor_pr_fN[0])
        lat.append(coor_pr_fN[1])
        
    interface_pix = pd.DataFrame({'pr':PRS, 'row':rows, 'col':cols, 'signature':SS})

    return lon, lat, interface_pix


**STORE THE INFORMATION**

In [21]:
def Save_geojson(paths,shapefile, imagename, lon, lat):
    
    # Save into the geojson file
    
    geojson = PRIS_tools_v2.coor_to_geojson(lon, lat)
    
    filename = imagename.split('.')[0] + '_' + 'profiles_'+   '_'.join(shapefile.split('_')[2:4]) + '.geojon'
    
    dir_store = os.path.join(paths['SDS'], 'derivative')
    
    if not os.path.exists(dir_store):
        os.mkdir(dir_store)
    
    PRIS_tools_v2.save_geojson(dir_store, filename, geojson)

In [22]:
def SaveSubPixel_Txt(paths, shapefile ,imagename, lon, lat):
    
    dir_img =  os.path.join(paths['PanSharp_Square_Cropped'], imagename)
    raster = gdal.Open(dir_img)
    geoT = raster.GetGeoTransform()
    minx = geoT[0]
    maxy = geoT[3]
    
    upper_left = [minx, maxy]
    
    filename = imagename.split('.')[0] + '_' + 'profiles_'+   '_'.join(shapefile.split('_')[2:4]) + '.txt'
    
    dir_store = os.path.join(paths['SDS'], 'derivative')
    
    if not os.path.exists(dir_store):
        os.mkdir(dir_store)
        
    dir_fileout = os.path.join(paths['SDS'], 'derivative', filename)
    
    fileout = open(dir_fileout, "w")

    for lo, la in zip(lon, lat):
            
        sub_col = (lo - upper_left[0])/geoT[1]
        sub_row = (upper_left[1] - la)/geoT[1]
        fileout.write('{:9.3f} {:9.3f}\n'.format(sub_row, sub_col))
                         
    fileout.close()


**MAIN FUNCTION**

In [23]:
def SubPixProfiles(paths):
    
    # Load the images we have to analyze each one
    
    imgs = [f for f in os.listdir(paths['PanSharp_Square_Cropped']) if f.endswith('tiff')]
    
    
    for imagename in imgs:
        
        date = imagename.split('_')[1]
        
        if date == '20201015':
            
            continue
        
        print('Analyzing imgage: ', str(imagename), flush = True)
      
        
        dir_img =  os.path.join(paths['PanSharp_Square_Cropped'], imagename)
        print('leo imagen')
        arr, arr_rgb = PRIS_img_v2.numpy_from_tiff(dir_img, True)
        print('he leido imagen')
        img = gdal.Open(dir_img)
        GeoT = img.GetGeoTransform()
        proj = osr.SpatialReference(wkt=img.GetProjection())
        crs = pyproj.CRS("EPSG:" + proj.GetAttrValue('AUTHORITY',1))

        
        # CREATE THE RASTER GRID GEODATAFRAME
        
        #grid = GridRaster(dir_img, crs)

        dir_grids = os.path.join(paths['scene'] , beach, 'QGIS', 'Img_grid')
        gridName = [f for f in os.listdir(dir_grids) if date in f and 'Upsampling' not in f][0]
        grid = gpd.read_file(os.path.join(dir_grids, gridName))
        
        # Image Bounding Box
        
        xmin, xmax, ymin, ymax, north = ImageBB(paths, date)
        img_BB = Polygon([[xmin,ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
        
        ## LOAD THE PROFILES
        
        shapefiles = [f for f in os.listdir(paths['Profiles']) if f.endswith('.shp') and date in f]
        
        # if there is more than one profile per date (inclination study) we must iterate throught
        
        for shapefile in shapefiles:
            
            print('SDS for profiles: ', shapefile)
            
            filename = imagename.split('.')[0] + '_' + 'profiles_'+   '_'.join(shapefile.split('_')[2:4]) + '.txt'

            #if os.path.exists(os.path.join(paths['SDS'], 'derivative', filename)):
                #print('Already calculated')
                #continue
        
            profile_path = os.path.join(paths['Profiles'],shapefile) 

            if os.path.exists(profile_path):
                print('Load the profiles')
                profiles = gpd.read_file(profile_path, encoding="utf-8")
                crs_pr = profiles.crs
            else:
                print('No profiles')
                
            if '90_0' in shapefile:
                plot = False
            else:
                plot = False

            # ITERATE THROUGH THE PROFILES
            if '90_0' in shapefile:
                lon, lat, interface_pix = AnaliziyngProfiles(profiles,date, grid,arr, img_BB, GeoT, plot)
            
            # SAVE THE INTERFACE_PIX DATAFRAME
            
            # This information is saved only for the k-means shoreline (perp pr)
            
            if '90_0' in shapefile:
                
                name_csv = 'InterfacePix_' + date +'_'+('_').join(shapefile.split('.')[0].split('_')[-3:-1]) + '.csv'
        
                interface_pix.to_csv(os.path.join(paths['Interface_pix_csv'], name_csv), index = False)

            # SAVE THE SHORELINE

            Save_geojson(paths,shapefile, imagename, lon, lat)
            SaveSubPixel_Txt(paths,shapefile, imagename, lon, lat)


**TRY THE ALGORITHM**

In [24]:
path0 = '/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/'
scene = 'FR'

beaches = [f for f in os.listdir(os.path.join(path0, 'SCENES', scene)) if 'FR03' in f] #if '01' in f

for beach in beaches:
    
    print(beach)
    
    paths = PRIS_tools_v2.generate_structure(path0, scene, beach)
    
    dir_inter_Rat_txtPlot = os.path.join(paths['scene'], beach, 'Results', 'Interface_pix', 'txtPlot')
    
    if not os.path.exists(dir_inter_Rat_txtPlot):
        os.makedirs(dir_inter_Rat_txtPlot)
        
    
    paths['Interface_pix_txtPlot'] = dir_inter_Rat_txtPlot
    
    SubPixProfiles(paths)

FR03
Analyzing imgage:  FR_20210312_0101_0101_01010_00101.tiff
leo imagen
he leido imagen
SDS for profiles:  FR_20210312110747_90_0_pr.shp
Load the profiles
 Analyzing PR366